# Coursework for PA4611

This notebook contains coursework for PA4611 "Computational Fluid and Phase-Space Flows" at University of Leicester.

This piece of course is the second half of the second, and final, computational project. It is focussed on computational hydrodynamics, in this case using a grid solver to follow behaviour in 2 dimensions. Please refer to the associated slides for some key science and theory.

## Ulula - a 2D grid solver
For this piece of coursework you do not need to write your own grid-solver for hydrodynamics. Instead, you will be using one that has already written. It is called Ulula, and was written by Benedikt Diemer (University of Maryland).

The version you will be using is one that I have adapted slightly to do the things I want it to do, but the overwhelming majority of the code is Diemer's. 

You should download the code from the course Github page: https://bitbucket.org/PaulMcMillan-Astro/ulula/src/master/, and there is documentation for most of it here: https://bdiemer.bitbucket.io/ulula/index.html

## Installation

The code is in the course Github repository and can be copied from there. The main ways to do that would be:

1. Using the command line: 
    - If you have not used git to get a copy of the course repository then `git clone https://github.com/PaulMcMillan-Astro/PA4611-Workshops.git` or, if you really know what you're doing `git clone git@github.com:PaulMcMillan-Astro/PA4611-Workshops.git`
    - If you have already cloned the repository then you can just `git pull` to get the latest version.
2. Using the web interface: go to the repository and download the zip file. (You will need to click on the button saying 'Code' to download the zip file.)
    - You will need to unzip the file and move the directory to where you want it to be (ideally the directory this notebook is in).
3. From Blackboard. I have put the zip file `ulula.zip`, which I will aim to keep fully up-to-date. You should download this and unzip it in the directory where you want to work.

You may need to use the lines of code using `sys.path' to make sure the notebook can use the Ulula code.

To make Ulula work you need the usual packages (numpy, scipy & matplotlib) and h5py and six. You can install these using conda install or pip install.

## The final project - The Kelvin-Helmholtz instability

The final project is to investigate the Kelvin-Helmholtz instability. This is a hydrodynamic instability that occurs when there is a velocity shear between two fluids. It is a common feature in astrophysical flows.

You will use the Ulula code to investigate how this instability develops in a 2D grid. The code allows you to set-up and run the code in a variety of ways, and you should experiment with different parameters to see how the instability develops.

# Warning

This notebook is not filled out fully yet, but you can use it to look at the code you'll be using

# Warning

In [1]:
import sys

# Gives the path of the ulula. Here it is assumed to a directory 
# in the same directory as this notebook. If you change that, change this line.
sys.path.insert(0,'./ulula') 


In [2]:
# The key bits of ulula for this project
import ulula.setups.kelvin_helmholtz as setup_kh
import ulula.run as ulula_run
import ulula.simulation as ulula_sim

## Running a simulation (example settings)

Let's start by showing you want can be produced by running a simulation. The simulation below starts with a simple Kelvin-Helmholtz-unstable system, then evolves it until t=3 on a 200x200 grid using no reconstruction, Euler integration, and $\alpha_{cfl}$ value of 0.5, plotting the density and x-velocity every 0.5 time units. The fluid layers both have adiabatic index $\gamma=5/3$, so $P\propto \rho^{5/3}$.

The plots are placed in the directory plots/example (which is created automatically if not already in place). Each plot is given the suffix CE (which I chose because it is a 'const' reconstruction and Euler timestepping). You should choose your own plot directory and suffix for your own plots.

In [3]:
setup_basic = setup_kh.SetupKelvinHelmholtz(sharp_ics=False, n_waves=1, rho1=2.0, velocity_difference=1.0,)
hs_basic = ulula_sim.HydroScheme(reconstruction = 'const', time_integration= 'euler', cfl = 0.5)

kwargs_basic = dict(tmax = 3.0, nx = 200, plot_time = 0.5, q_plot = ['DN', 'VX'], plot_dir = 'plots/example', plot_suffix='CE')
ulula_run.run(setup_basic, hydro_scheme=hs_basic, **kwargs_basic)

Setting up Kelvin-Helmholtz instability
running...
Grid setup 200 x 200, dimensions x = [0.00e+00 .. 1.00e+00] y =  [0.00e+00 .. 1.00e+00]
Timestep   100, dt = 9.84e-04, t = 9.84e-02
Timestep   200, dt = 9.80e-04, t = 1.97e-01
Timestep   300, dt = 9.80e-04, t = 2.95e-01
Timestep   400, dt = 9.82e-04, t = 3.93e-01
Timestep   500, dt = 9.83e-04, t = 4.91e-01
Timestep   600, dt = 9.86e-04, t = 5.89e-01
Timestep   700, dt = 9.89e-04, t = 6.88e-01
Timestep   800, dt = 9.92e-04, t = 7.87e-01
Timestep   900, dt = 9.97e-04, t = 8.86e-01
Timestep  1000, dt = 1.00e-03, t = 9.86e-01
Timestep  1100, dt = 1.01e-03, t = 1.09e+00
Timestep  1200, dt = 1.01e-03, t = 1.19e+00
Timestep  1300, dt = 1.02e-03, t = 1.29e+00
Timestep  1400, dt = 1.02e-03, t = 1.39e+00
Timestep  1500, dt = 1.03e-03, t = 1.49e+00
Timestep  1600, dt = 1.04e-03, t = 1.60e+00
Timestep  1700, dt = 1.05e-03, t = 1.70e+00
Timestep  1800, dt = 1.05e-03, t = 1.81e+00
Timestep  1900, dt = 1.06e-03, t = 1.91e+00
Timestep  2000, dt = 1.07

The results there are very blurry, and don't look at all like the results that we would hope for. Fortunately we have various parameters of the simulation that we can alter to improve/change the simulation. I will explain them below, but you should also look at the documentation, and into the ulula code itself, to see what else you can do.

One useful thing to do is to look at the documentation string for each function used above. I will show a simple way of doing that below. 

## The functions and their parameters
### setup_kh.SetupKelvinHelmholtz

This is used to set up the simulation given the choice of parameters. When the simulation actually runs it uses this 'object' (called setup_basic for the example above) to set up the particular simulation.

In [4]:
# Get the help string for the setup code
?setup_kh.SetupKelvinHelmholtz
# Or you can do it this way:
#help(setup_kh.SetupKelvinHelmholtz)

Init signature:
setup_kh.SetupKelvinHelmholtz(
    sharp_ics=False,
    n_waves=1,
    rho1=2.0,
    velocity_difference=1.0,
)
Docstring:     
Kelvin-Helmholtz instability

The KH instability forms at the interface between two fluids that are moving past each other.
The user can choose between a setup where the interface is infinitely sharp and the smooth
ICs of Robertson et al. 2010. In the sharp case, the instability is seeded by grid noise, but
we still add a small velocity perturbation to obtain more well-defined behavior. The smooth
version is recommended as it leads to a more physical test case.

Parameters
-----------------------------------------------------------------------------------------------
sharp_ics: bool, default False
        Use sharp boundary between fluids instead of smoothed ICs.
n_waves: int, default 1
        Number of wave periods in the domain. The number of periods that can be resolved depends on the resolution.
rho1: float, default 2.0
        Density of 

You can choose whether you want a sharp divide between the fluids initially, how many waves you want in the seed perturbation, and the density of the denser fluid and the relative velocities of the fluid.

### ulula_sim.HydroScheme
Defines how the simulation will solve the fluid-dynamical equations. For the example above I have called this  hs_basic

In [5]:
?ulula_sim.HydroScheme

Init signature:
ulula_sim.HydroScheme(
    reconstruction='const',
    limiter='minmod',
    riemann='hll',
    time_integration='euler',
    cfl=0.8,
)
Docstring:     
Container class for hydro algorithms

Parameters
-----------------------------------------------------------------------------------------------
reconstruction: string
        Reconstruction algorithm; see listing for valid choices
limiter: string
        Slope limiter algorithm; see listing for valid choices
riemann: string
        Riemann solver; see listing for valid choices
time_integration: string
        Time integration scheme; see listing for valid choices
cfl: float
        CFL number (must be between 0 and 1); determines the timestep as CFL number times cell size
        divided by the maximum signal speed in the domain
File:           ~/code/PA4611-Workshops/ulula/simulation.py
Type:           type
Subclasses:     

The options are:  
reconstruction can be 'const' or 'linear', i.e., either constant within each bin or with a linear reconstruction within each bin.  
limiter can be 'none', 'minmod', 'vanleer' or 'mc'. These are only used if there is a linear reconstruction.  
riemann can only be 'hll' so no need to change that.  
time_integration can be 'euler', 'hancock' or 'hancock_cons'
cfl should be a value between $0$ and $1$ because otherwise the

 (Note that in each case the parameters need to be given as strings, i.e. in quote marks)

### ulula_run.run

This is the command that runs the simulation itself. It takes the setup and HydroScheme objects as arguments, and also a number of other arguments...


In [6]:
?ulula_run.run

Signature:
ulula_run.run(
    setup,
    hydro_scheme=None,
    nx=200,
    tmax=1.0,
    max_steps=None,
    print_step=100,
    restart_file=None,
    output_step=None,
    output_time=None,
    output_suffix='',
    plot_step=None,
    plot_time=None,
    plot_ics=True,
    plot1d=False,
    save_plots=True,
    plot_dir='.',
    plot_suffix='',
    plot_file_ext='png',
    plot_dpi=300,
    movie=False,
    movie_length=4.0,
    movie_fps=25,
    movie_dpi=200,
    **kwargs,
)
Docstring:
Runtime environment for Ulula.

This function takes a given problem setup and other user-defined parameters and executes the
hydro solver. Depending on user choices, it can also produces output files, plots, and movies.
Customizations that are implemented in the setup class (e.g., which variables to plot with
which colormaps) are automatically routed to the respective plotting routines.

Parameters
-----------------------------------------------------------------------------------------------
setup

# Your work

What I want you to do is, based on the lectures explaining grid hydrodynamics, adjust this code to produce a simulation that shows the Kelvin Helmholtz instability in a way that appears to be physically realistic. 

Then, below, show how adjusting parameters of the simulations affects the results. This includes the setup, the hydro scheme or the number of points in the grid. This will almost certainly be in graphical form unless you want to start changing the ulula code itself. It would be impressive if you do that to quantify your results, but is in no way a necessary thing to do.

I would expect you to clearly describe the effect of changing 3 or 4 of the possible parameters of the simulations. This requires you to show plots and write clearly about what changes you made, how they affect the results and why you think that is the case.

To include figures when you hand in this notebook you will have to ensure that when I run the notebook, the files are produced in a folder within the same one as the notebook. An example of this can be could be seen in the example above. You then need to use those figures in the notebook. You can do that as shown in the cell below.

You should display these plots in context for your write-up. The best method for doing this is to use a simple bit of HTML like that used below. This will allow you to display the plots in a way that is easy to see and understand. E.g.:

The initial conditions of the simulation look like this:  

<img src="plots/example/ulula_kh_time_0.0000CE.png" alt="Put alt text here if you want" width="400"/>

Note that the figure did not exist until the code above was run.


An alternative method (which I don't recommend but acknowledge I should mention) using pure Markdown syntax is the following, but the problem with this is that the images are not resized, and so can be very large. E.g.:

The initial conditions of the simulation look like this:  

![alt text](plots/example/ulula_kh_time_0.0000CE.png)

This is deliberately open-ended. I will mark it based on how well you investigate and explain the effects of changing the parameters of the simulation. One important question: Why does it need to seed a particular wavelength. Which wavelengths are affected by the Kelvin-Helmholtz instability?